In [13]:
!pip install transformers==4.36.2
!pip install --upgrade diffusers[torch]==0.25.1
!pip install accelerate==0.25.0
# # !pip install xformers==0.0.23.post1

In [14]:
offset = 0
last_image_index = None#None if don't know the number of images
QF = 25
file_path = '/kaggle/input/image-description/Image_description.txt'

In [15]:
%load_ext autoreload
%autoreload 2
from diffusers import AutoencoderKL
from diffusers import UNet2DConditionModel
from transformers import CLIPTextModel, CLIPTokenizer

import os
import PIL
from PIL import Image, ImageOps
import requests
import torch
import matplotlib.pyplot as plt
import numpy as np

import gc
import json
import time
import torch
import requests
from tqdm import tqdm
from io import BytesIO
from diffusers import StableDiffusionImg2ImgPipeline
import torchvision.transforms as T
from typing import Union, List, Optional, Callable

to_pil = T.ToPILImage()

from diffusers import AutoencoderKL
from diffusers import UNet2DConditionModel

from transformers import CLIPTextModel, CLIPTokenizer
import requests
from io import BytesIO

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
def init_seeds(seed):
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  os.environ["PYTHONHASHSEED"] = str(seed)
  os.environ['PYTORCH_CUDA_ALLOC_CONF']='max_split_size_mb:224'

totensor = T.ToTensor()
topil = T.ToPILImage()

hyperparams = {
    'SEED' : 9222,
    'STRENGTH' : 0.7,
    'GUIDANCE_SCALE' : 7.5,
    'NUM_DENOISING_STEPS' : 4,
    'NUM_DENOISING_STEPS_FINAL':50,
    'EPS' : 0.1,
    'STEP_SIZE' : 0.006,
    'ITERS' : 1,
    'CLAMP_MIN' : -1,
    'CLAMP_MAX' : 1,
    'GRAD_REPS' : 1,
    'PROMPT_ATTACK': ''
}

init_seeds(hyperparams['SEED'])

In [17]:
def recover_image(image, init_image, mask, background=False):
    image = totensor(image)
    mask = totensor(mask)
    init_image = totensor(init_image)
    if background:
        result = mask * init_image + (1 - mask) * image
    else:
        result = mask * image + (1 - mask) * init_image
    return topil(result)

def preprocess(image):
    w, h = image.size
    w, h = map(lambda x: x - x % 32, (w, h))  # resize to integer multiple of 32
    image = image.resize((w, h), resample=Image.LANCZOS)
    image = np.array(image).astype(np.float32) / 255.0
    image = image[None].transpose(0, 3, 1, 2)
    image = torch.from_numpy(image)
    return 2.0 * image - 1.0

def prepare_image(image):
    image = np.array(image.convert("RGB"))
    image = image[None].transpose(0, 3, 1, 2)
    image = torch.from_numpy(image).to(dtype=torch.float32) / 127.5 - 1.0

    return image[0]

In [18]:
# , KarrasDiffusionSchedulers
# Load scheduler and models

device = 'cuda'
unet = UNet2DConditionModel.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    subfolder="unet",
    torch_dtype=torch.float16,
).to(device)
# text_encoder = CLIPTextModel.from_pretrained(
#     "openai/clip-vit-large-patch14",
#     subfolder="unet", 
# )
tokenizer = CLIPTokenizer.from_pretrained(
    "openai/clip-vit-large-patch14",
    torch_dtype = torch.float16    
    )

text_encoder = CLIPTextModel.from_pretrained(
    "openai/clip-vit-large-patch14",
        torch_dtype = torch.float16
    
    ).to(device)

vae = AutoencoderKL.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    subfolder="vae",
    torch_dtype=torch.float16
).to(device)


# unet.requires_grad_(False)
# vae.requires_grad_(False)

unet.enable_gradient_checkpointing()
text_encoder.gradient_checkpointing_enable()
vae._set_gradient_checkpointing(module=vae.encoder, value=True)
vae._set_gradient_checkpointing(module=vae.decoder, value=True)
# unet.enable_xformers_memory_efficient_attention()

# from diffusers import DiffusionPipel                                                                                                                     ine, DDIMScheduler
from diffusers import DDIMScheduler
scheduler = DDIMScheduler()


In [19]:
preprocess_res = T.Compose([
    T.Resize(256),
    T.CenterCrop(224),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

preprocess_tensor = T.Compose([
    T.ToTensor(),
])

preprocess_tensor_pil= T.ToPILImage()

In [20]:
def download_image(url):
    try:
        response = requests.get(url)
        init_image = Image.open(BytesIO(response.content)).convert("RGB")
        resize = T.transforms.Resize(512)
        center_crop = T.transforms.CenterCrop(512)
        init_image = center_crop(resize(init_image))

        return init_image
    except (requests.exceptions.RequestException, PIL.UnidentifiedImageError) as e:
    # Handle the exception here
        print(f"Error downloading/processing image: {e}")
    # You can return None, a default image, or log the error
        return None  # Example: return None if download fails

def read_image_prompt(file_path):
    with open(file_path, 'r') as file:
        text = file.read()
    sentences = text.split('\n')
    image_url = []
    prompt = []
    image_description = []
    for i, sentence in enumerate(sentences):
        if i%3==0:
            image_url.append(sentence)
        elif i%3==1:
            prompt.append(sentence)
        elif i%3==2:
            image_description.append(sentence)
            

    return (image_url, prompt, image_description)


image_url, prompt, _ = read_image_prompt(file_path)
image_url = image_url[:-1]

#take only some images  #-------------------------------------------------------------->>>>>>>>>>>>updates here
if last_image_index == None:
    image_url = image_url[offset:]
    prompt = prompt[offset:]
    
else:
    image_url = image_url[offset:last_image_index]
    prompt = prompt[offset:last_image_index]


In [21]:
# A differentiable version of the forward function of the inpainting stable diffusion model! See https://github.com/huggingface/diffusers
# To understand https://huggingface.co/docs/diffusers/using-diffusers/write_own_pipeline

#careful on gradient direction
def attack_forward(
        prompt: Union[str, List[str]],
        image: Union[torch.FloatTensor, Image.Image],
        height: int = 512,
        width: int = 512,
        num_inference_steps: int = 50,
        guidance_scale: float = 7.5,
    ):
        text_inputs = tokenizer(
            prompt,
            padding="max_length",
            max_length=tokenizer.model_max_length,
            return_tensors="pt",
        )
        text_input_ids = text_inputs.input_ids

        text_embeddings = text_encoder(text_input_ids.to(device))[0]

        uncond_tokens = [""]
        #unconditional text embeddings for the padding token
        max_length = text_input_ids.shape[-1]
        uncond_input = tokenizer(
            uncond_tokens,
            padding="max_length",
            max_length=max_length,
            truncation=True,
            return_tensors="pt",
        )
        uncond_embeddings = text_encoder(uncond_input.input_ids.to(device))[0]
        seq_len = uncond_embeddings.shape[1]
        text_embeddings = torch.cat([uncond_embeddings, text_embeddings])
        text_embeddings = text_embeddings.detach()

        num_channels_latents = vae.config.latent_channels

        # latents_shape = (1 , num_channels_latents, height // 8, width // 8)
        # latents = torch.randn(latents_shape, device=self.device, dtype=text_embeddings.dtype)
        image_latents = vae.encode(image).latent_dist.sample()
        image_latents = 0.18215 * image_latents
        # image_latents = torch.cat([image_latents] * 2)
        
        latents = image_latents * scheduler.init_noise_sigma
        scheduler.set_timesteps(num_inference_steps)
        timesteps_tensor = scheduler.timesteps.to(device)

        for i, t in enumerate(timesteps_tensor):
            # expand the latents if we are doing classifier-free guidance to avoid doing two forward passes.
            latent_model_input = torch.cat([latents] * 2)
            latent_model_input = scheduler.scale_model_input(latent_model_input, timestep=t)
            # latent_model_input = torch.cat([latent_model_input, image_latents], dim=1)
            noise_pred = unet(latent_model_input, t, encoder_hidden_states=text_embeddings).sample
            
            noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
            #use the noise prediction of both image and text
            noise_pred = noise_pred_uncond + guidance_scale * (noise_pred_text - noise_pred_uncond)
            # compute the previous noisy sample x_t -> x_t-1
            latents = scheduler.step(noise_pred, t, latents).prev_sample #why
            
            del latent_model_input
            del noise_pred_uncond, noise_pred_text
            del noise_pred
            torch.cuda.empty_cache()
            
        latents = 1 / 0.18215 * latents #get the latents
        image = vae.decode(latents).sample
        return image


def compute_grad(X, cur_image, prompt,  **kwargs):
    torch.set_grad_enabled(True)
    cur_image = cur_image.clone()
    cur_image.requires_grad_()
    image = attack_forward(
                               image=cur_image,
                               prompt=prompt,
                               **kwargs)
#     loss = (image_nat - target_image).norm(p=2)
#     loss = lpips(image.clamp(-1,1), X)
    loss = image.norm(p=2)
    grad = torch.autograd.grad(loss, [cur_image])[0]
    
    return grad, loss.item(), image.data.cpu()

def super_l2( X, prompt, step_size, iters, eps, clamp_min, clamp_max, grad_reps = 5, **kwargs):
    X_adv = X.clone()
    iterator = tqdm(range(iters))

    for i in iterator:

        all_grads = []
        losses = []

        for i in range(grad_reps): #Why do this for same image, prompt and parameters?
            c_grad, loss, last_image = compute_grad(X, X_adv, prompt, **kwargs)
            all_grads.append(c_grad)
            losses.append(loss)
        
        grad = torch.stack(all_grads).mean(0)

        iterator.set_description_str(f'AVG Loss: {np.mean(losses):.3f}')

        l = len(X.shape) - 1   #why?
        grad_norm = torch.norm(grad.detach().reshape(grad.shape[0], -1), dim=1).view(-1, *([1] * l))
        grad_normalized = grad.detach() / (grad_norm + 1e-10)

        # actual_step_size = step_size - (step_size - step_size / 100) / iters * i
        actual_step_size = step_size
        X_adv = X_adv - grad_normalized * actual_step_size

        d_x = X_adv - X.detach()  #Perturbations
        d_x_norm = torch.renorm(d_x, p=2, dim=0, maxnorm=eps)   #Why?
        X_adv.data = torch.clamp(X + d_x_norm, clamp_min, clamp_max)
    

        torch.cuda.empty_cache()

    return X_adv, last_image

def super_linf(X,jpeg_model, prompt, step_size, iters, eps, clamp_min, clamp_max, grad_reps = 5, **kwargs):
    X_adv = X.clone()
    iterator = tqdm(range(iters))

    for i in iterator:
        all_grads_jpeg = []
        losses = []
        
        QFs = [25, 50, 75, 100]
        
        for QF in QFs:
            X_jpeg = jpeg_model(X_adv.float(), QF).half()
            all_grads = []

            for i in range(grad_reps):
                c_grad, loss, last_image = compute_grad(X, X_jpeg, prompt, **kwargs)
                all_grads.append(c_grad)
                losses.append(loss)
            grad = torch.stack(all_grads).mean(0)
            iterator.set_description_str(f'AVG Loss: {np.mean(losses):.3f}')
            all_grads_jpeg.append(grad)
            
        grad = torch.stack(all_grads_jpeg).mean(0)
        
        actual_step_size = step_size - (step_size - step_size / 100) / iters * i
#         actual_step_size = step_size

        X_adv = X_adv - grad.detach().sign() * actual_step_size

        X_adv = torch.minimum(torch.maximum(X_adv, X - eps), X + eps)
        X_adv.data = torch.clamp(X_adv, min=clamp_min, max=clamp_max)


    torch.cuda.empty_cache()

    return X_adv, last_image, np.mean(losses)

In [22]:
def gen_image(init_image, adv_image, path_to_save, prompt,  QF:int = 60):

    (height, width) = (512, 512)
    with torch.no_grad():

        adv_image.save(f'{path_to_save}/adv_image.png')
        init_image.save(f'{path_to_save}/original_image.png')
        num_compressions = (95-QF)//10
        jump = 10

        fig, ax = plt.subplots(nrows=2, ncols=num_compressions+2, figsize=(20,6))
        ax[0,0].imshow(init_image)
        ax[0,1].imshow(adv_image)
        # Play with these for improving generated image quality
        # SEED = np.random.randint(low=0, high=10000)

        STRENGTH = hyperparams['STRENGTH']
        GUIDANCE = hyperparams['GUIDANCE_SCALE']
        NUM_STEPS = hyperparams['NUM_DENOISING_STEPS_FINAL']
        SEED  = hyperparams['SEED']
        with torch.autocast('cuda'):
            torch.manual_seed(SEED)
            image_nat = pipe_img2img(prompt=prompt, image=init_image, strength=STRENGTH, guidance_scale=GUIDANCE, num_inference_steps=NUM_STEPS).images[0]
            torch.manual_seed(SEED)
            image_adv = pipe_img2img(prompt=prompt, image=adv_image, strength=STRENGTH, guidance_scale=GUIDANCE, num_inference_steps=NUM_STEPS).images[0]

        image_nat.save(f'{path_to_save}/gen_original_image.png')
        image_adv.save(f'{path_to_save}/gen_adversarial.png')
        ax[1,0].imshow(image_nat)
        ax[1,1].imshow(image_adv)
        ax[0,0].set_title('Source', fontsize=16)
        ax[0,1].set_title('Adv.', fontsize=16)

        for i in range(num_compressions):

            jpeg = DiffJPEG(height=height, width=width, differentiable=True).to('cuda')
            compressed_img = jpeg(preprocess_tensor(adv_image).unsqueeze(0).to('cuda') , quality=QF)

            preprocess_tensor_pil(compressed_img[0]).save(f"{path_to_save}/adv_image_compress_{QF}.png")
            jpg_image = Image.open(f"{path_to_save}/adv_image_compress_{QF}.png")
            with torch.autocast('cuda'):
                torch.manual_seed(SEED)
                image_adv_jpeg = pipe_img2img(prompt=prompt, image=jpg_image, strength=STRENGTH, guidance_scale=GUIDANCE, num_inference_steps=NUM_STEPS).images[0]
                image_adv_jpeg.save(f'{path_to_save}/gen_adv_image{QF}.png')

                ax[0,i+2].imshow(jpg_image)
                ax[1,i+2].imshow(image_adv_jpeg)
                ax[0,i+2].set_title(f'QF: {QF}.', fontsize=16)
            QF = QF + jump


        for i in range(2):
            for j in range(num_compressions):
                ax[i][j].grid(False)
                ax[i][j].axis('off')

        fig.suptitle(f"Prompt: {prompt}", fontsize=20)
        fig.tight_layout()
#         plt.show(plt.savefig(f"{path_to_save}/Result.png", bbox_inches='tight'))
        plt.savefig(f"{path_to_save}/Result.png", bbox_inches='tight')

        
def save_dict_to_text_file(dictionary, file_path, total_time, loss):
    """
    Convert a dictionary to a JSON-formatted string and save it to a text file.

    Parameters:
    - dictionary: The dictionary to be saved.
    - file_path: The target location and filename for the text file.
    """
    try:
        with open(f'{file_path}/config.txt', 'w') as file:
            json.dump(dictionary, file, indent=4)
            file.write(f'total time: {total_time}')
            for i, val in enumerate(loss):
                file.write(f'Loss Sample {i}: {val}')

        print(f"Dictionary successfully saved to {file_path}")
    except Exception as e:
        print(f"Error saving dictionary to {file_path}: {e}")
    
    


In [23]:
# We didn't write this ourselves but I don't remember from where we took this will find it's source and update


# Standard libraries
import itertools
import numpy as np
# PyTorch
import torch
import torch.nn as nn

y_table = np.array(
    [[16, 11, 10, 16, 24, 40, 51, 61], [12, 12, 14, 19, 26, 58, 60,
                                        55], [14, 13, 16, 24, 40, 57, 69, 56],
     [14, 17, 22, 29, 51, 87, 80, 62], [18, 22, 37, 56, 68, 109, 103,
                                        77], [24, 35, 55, 64, 81, 104, 113, 92],
     [49, 64, 78, 87, 103, 121, 120, 101], [72, 92, 95, 98, 112, 100, 103, 99]],
    dtype=np.float32).T

y_table = nn.Parameter(torch.from_numpy(y_table))
#
c_table = np.empty((8, 8), dtype=np.float32)
c_table.fill(99)
c_table[:4, :4] = np.array([[17, 18, 24, 47], [18, 21, 26, 66],
                            [24, 26, 56, 99], [47, 66, 99, 99]]).T
c_table = nn.Parameter(torch.from_numpy(c_table))


def diff_round(x):
    """ Differentiable rounding function
    Input:
        x(tensor)
    Output:
        x(tensor)
    """
    return torch.round(x) + (x - torch.round(x))**3


def quality_to_factor(quality):
    """ Calculate factor corresponding to quality
    Input:
        quality(float): Quality for jpeg compression
    Output:
        factor(float): Compression factor
    """
    if quality < 50:
        quality = 5000. / quality
    else:
        quality = 200. - quality*2
    return quality / 100.


class y_dequantize(nn.Module):
    """ Dequantize Y channel
    Inputs:
        image(tensor): batch x height x width
        factor(float): compression factor
    Outputs:
        image(tensor): batch x height x width

    """
    def __init__(self):
        super(y_dequantize, self).__init__()
        self.y_table = y_table

    def forward(self, image, factor):
        return image * (self.y_table * factor)


class c_dequantize(nn.Module):
    """ Dequantize CbCr channel
    Inputs:
        image(tensor): batch x height x width
        factor(float): compression factor
    Outputs:
        image(tensor): batch x height x width

    """
    def __init__(self):
        super(c_dequantize, self).__init__()
        self.c_table = c_table

    def forward(self, image , factor):
        return image * (self.c_table * factor)


class idct_8x8(nn.Module):
    """ Inverse discrete Cosine Transformation
    Input:
        dcp(tensor): batch x height x width
    Output:
        image(tensor): batch x height x width
    """
    def __init__(self):
        super(idct_8x8, self).__init__()
        alpha = np.array([1. / np.sqrt(2)] + [1] * 7)
        self.alpha = nn.Parameter(torch.from_numpy(np.outer(alpha, alpha)).float())
        tensor = np.zeros((8, 8, 8, 8), dtype=np.float32)
        for x, y, u, v in itertools.product(range(8), repeat=4):
            tensor[x, y, u, v] = np.cos((2 * u + 1) * x * np.pi / 16) * np.cos(
                (2 * v + 1) * y * np.pi / 16)
        self.tensor = nn.Parameter(torch.from_numpy(tensor).float())

    def forward(self, image):

        image = image * self.alpha
        result = 0.25 * torch.tensordot(image, self.tensor, dims=2) + 128
        result.view(image.shape)
        return result


class block_merging(nn.Module):
    """ Merge pathces into image
    Inputs:
        patches(tensor) batch x height*width/64, height x width
        height(int)
        width(int)
    Output:
        image(tensor): batch x height x width
    """
    def __init__(self):
        super(block_merging, self).__init__()

    def forward(self, patches, height, width):
        k = 8
        batch_size = patches.shape[0]
        image_reshaped = patches.view(batch_size, height//k, width//k, k, k)
        image_transposed = image_reshaped.permute(0, 1, 3, 2, 4)
        return image_transposed.contiguous().view(batch_size, height, width)


class chroma_upsampling(nn.Module):
    """ Upsample chroma layers
    Input:
        y(tensor): y channel image
        cb(tensor): cb channel
        cr(tensor): cr channel
    Ouput:
        image(tensor): batch x height x width x 3
    """
    def __init__(self):
        super(chroma_upsampling, self).__init__()

    def forward(self, y, cb, cr):
        def repeat(x, k=2):
            height, width = x.shape[1:3]
            x = x.unsqueeze(-1)
            x = x.repeat(1, 1, k, k)
            x = x.view(-1, height * k, width * k)
            return x

        cb = repeat(cb)
        cr = repeat(cr)

        return torch.cat([y.unsqueeze(3), cb.unsqueeze(3), cr.unsqueeze(3)], dim=3)


class ycbcr_to_rgb_jpeg(nn.Module):
    """ Converts YCbCr image to RGB JPEG
    Input:
        image(tensor): batch x height x width x 3
    Outpput:
        result(tensor): batch x 3 x height x width
    """
    def __init__(self):
        super(ycbcr_to_rgb_jpeg, self).__init__()

        matrix = np.array(
            [[1., 0., 1.402], [1, -0.344136, -0.714136], [1, 1.772, 0]],
            dtype=np.float32).T
        self.shift = nn.Parameter(torch.tensor([0, -128., -128.]))
        self.matrix = nn.Parameter(torch.from_numpy(matrix))

    def forward(self, image):
        result = torch.tensordot(image + self.shift, self.matrix, dims=1)
        #result = torch.from_numpy(result)
        result.view(image.shape)
        return result.permute(0, 3, 1, 2)


class decompress_jpeg(nn.Module):
    """ Full JPEG decompression algortihm
    Input:
        compressed(dict(tensor)): batch x h*w/64 x 8 x 8
        rounding(function): rounding function to use
        factor(float): Compression factor
    Ouput:
        image(tensor): batch x 3 x height x width
    """
    def __init__(self, height, width, rounding=torch.round):
        super(decompress_jpeg, self).__init__()
        self.c_dequantize = c_dequantize()
        self.y_dequantize = y_dequantize()
        self.idct = idct_8x8()
        self.merging = block_merging()
        self.chroma = chroma_upsampling()
        self.colors = ycbcr_to_rgb_jpeg()

        self.height, self.width = height, width

    def forward(self, y, cb, cr , factor):
        components = {'y': y, 'cb': cb, 'cr': cr}
        for k in components.keys():
            if k in ('cb', 'cr'):
                comp = self.c_dequantize(components[k], factor)
                height, width = int(self.height/2), int(self.width/2)
            else:
                comp = self.y_dequantize(components[k], factor)
                height, width = self.height, self.width
            comp = self.idct(comp)
            components[k] = self.merging(comp, height, width)
            #
        image = self.chroma(components['y'], components['cb'], components['cr'])
        image = self.colors(image)

        image = torch.min(255*torch.ones_like(image),
                          torch.max(torch.zeros_like(image), image))
        return image/255


# Standard libraries
import itertools
import numpy as np
# PyTorch
import torch
import torch.nn as nn


class rgb_to_ycbcr_jpeg(nn.Module):
    """ Converts RGB image to YCbCr
    Input:
        image(tensor): batch x 3 x height x width
    Outpput:
        result(tensor): batch x height x width x 3
    """
    def __init__(self):
        super(rgb_to_ycbcr_jpeg, self).__init__()
        matrix = np.array(
            [[0.299, 0.587, 0.114], [-0.168736, -0.331264, 0.5],
             [0.5, -0.418688, -0.081312]], dtype=np.float32).T
        self.shift = nn.Parameter(torch.tensor([0., 128., 128.]))
        #
        self.matrix = nn.Parameter(torch.from_numpy(matrix))

    def forward(self, image):
        image = image.permute(0, 2, 3, 1)
        result = torch.tensordot(image, self.matrix, dims=1) + self.shift
    #    result = torch.from_numpy(result)
        result.view(image.shape)
        return result



class chroma_subsampling(nn.Module):
    """ Chroma subsampling on CbCv channels
    Input:
        image(tensor): batch x height x width x 3
    Output:
        y(tensor): batch x height x width
        cb(tensor): batch x height/2 x width/2
        cr(tensor): batch x height/2 x width/2
    """
    def __init__(self):
        super(chroma_subsampling, self).__init__()

    def forward(self, image):
        image_2 = image.permute(0, 3, 1, 2).clone()
        avg_pool = nn.AvgPool2d(kernel_size=2, stride=(2, 2),
                                count_include_pad=False)
        cb = avg_pool(image_2[:, 1, :, :].unsqueeze(1))
        cr = avg_pool(image_2[:, 2, :, :].unsqueeze(1))
        cb = cb.permute(0, 2, 3, 1)
        cr = cr.permute(0, 2, 3, 1)
        return image[:, :, :, 0], cb.squeeze(3), cr.squeeze(3)


class block_splitting(nn.Module):
    """ Splitting image into patches
    Input:
        image(tensor): batch x height x width
    Output:
        patch(tensor):  batch x h*w/64 x h x w
    """
    def __init__(self):
        super(block_splitting, self).__init__()
        self.k = 8

    def forward(self, image):
        height, width = image.shape[1:3]
        batch_size = image.shape[0]
        image_reshaped = image.view(batch_size, height // self.k, self.k, -1, self.k)
        image_transposed = image_reshaped.permute(0, 1, 3, 2, 4)
        return image_transposed.contiguous().view(batch_size, -1, self.k, self.k)


class dct_8x8(nn.Module):
    """ Discrete Cosine Transformation
    Input:
        image(tensor): batch x height x width
    Output:
        dcp(tensor): batch x height x width
    """
    def __init__(self):
        super(dct_8x8, self).__init__()
        tensor = np.zeros((8, 8, 8, 8), dtype=np.float32)
        for x, y, u, v in itertools.product(range(8), repeat=4):
            tensor[x, y, u, v] = np.cos((2 * x + 1) * u * np.pi / 16) * np.cos(
                (2 * y + 1) * v * np.pi / 16)
        alpha = np.array([1. / np.sqrt(2)] + [1] * 7)
        #
        self.tensor =  nn.Parameter(torch.from_numpy(tensor).float())
        self.scale = nn.Parameter(torch.from_numpy(np.outer(alpha, alpha) * 0.25).float() )

    def forward(self, image):
        image = image - 128
        result = self.scale * torch.tensordot(image, self.tensor, dims=2)
        result.view(image.shape)
        return result


class y_quantize(nn.Module):
    """ JPEG Quantization for Y channel
    Input:
        image(tensor): batch x height x width
        rounding(function): rounding function to use
        factor(float): Degree of compression
    Output:
        image(tensor): batch x height x width
    """
    def __init__(self, rounding):
        super(y_quantize, self).__init__()
        self.rounding = rounding
        self.y_table = y_table

    def forward(self, image, factor):
        image = image.float() / (self.y_table * factor)
        image = self.rounding(image)
        return image


class c_quantize(nn.Module):
    """ JPEG Quantization for CrCb channels
    Input:
        image(tensor): batch x height x width
        rounding(function): rounding function to use
        factor(float): Degree of compression
    Output:
        image(tensor): batch x height x width
    """
    def __init__(self, rounding):
        super(c_quantize, self).__init__()
        self.rounding = rounding
        self.c_table = c_table

    def forward(self, image, factor):
        image = image.float() / (self.c_table * factor)
        image = self.rounding(image)
        return image


class compress_jpeg(nn.Module):
    """ Full JPEG compression algortihm
    Input:
        imgs(tensor): batch x 3 x height x width
        rounding(function): rounding function to use
        factor(float): Compression factor
    Ouput:
        compressed(dict(tensor)): batch x h*w/64 x 8 x 8
    """
    def __init__(self, rounding=torch.round):
        super(compress_jpeg, self).__init__()
        self.l1 = nn.Sequential(
            rgb_to_ycbcr_jpeg(),
            chroma_subsampling()
        )
        self.l2 = nn.Sequential(
            block_splitting(),
            dct_8x8()
        )
        self.c_quantize = c_quantize(rounding=rounding)
        self.y_quantize = y_quantize(rounding=rounding)

    def forward(self, image, factor=1):
        y, cb, cr = self.l1(image*255)
        components = {'y': y, 'cb': cb, 'cr': cr}
        for k in components.keys():
            comp = self.l2(components[k])
            if k in ('cb', 'cr'):
                comp = self.c_quantize(comp, factor)
            else:
                comp = self.y_quantize(comp, factor)

            components[k] = comp

        return components['y'], components['cb'], components['cr']

class DiffJPEG(nn.Module):
    def __init__(self, height, width, differentiable=True):
        ''' Initialize the DiffJPEG layer
        Inputs:
            height(int): Original image hieght
            width(int): Original image width
            differentiable(bool): If true uses custom differentiable
                rounding function, if false uses standrard torch.round
            quality(float): Quality factor for jpeg compression scheme.
        '''
        super(DiffJPEG, self).__init__()
        if differentiable:
            rounding = diff_round
        else:
            rounding = torch.round
        self.compress = compress_jpeg(rounding=rounding)
        self.decompress = decompress_jpeg(height, width, rounding=rounding)

    def forward(self, x, quality=80):
        '''

        '''

        factor = quality_to_factor(quality)
        if factor == 0:
          return x

        y, cb, cr = self.compress(x, factor)
        recovered = self.decompress(y, cb, cr, factor)
        return recovered


    
# Standard libraries
import itertools
import numpy as np
# PyTorch
import torch
import torch.nn as nn

y_table = np.array(
    [[16, 11, 10, 16, 24, 40, 51, 61], [12, 12, 14, 19, 26, 58, 60,
                                        55], [14, 13, 16, 24, 40, 57, 69, 56],
     [14, 17, 22, 29, 51, 87, 80, 62], [18, 22, 37, 56, 68, 109, 103,
                                        77], [24, 35, 55, 64, 81, 104, 113, 92],
     [49, 64, 78, 87, 103, 121, 120, 101], [72, 92, 95, 98, 112, 100, 103, 99]],
    dtype=np.float32).T

y_table = nn.Parameter(torch.from_numpy(y_table))
#
c_table = np.empty((8, 8), dtype=np.float32)
c_table.fill(99)
c_table[:4, :4] = np.array([[17, 18, 24, 47], [18, 21, 26, 66],
                            [24, 26, 56, 99], [47, 66, 99, 99]]).T
c_table = nn.Parameter(torch.from_numpy(c_table))


def diff_round(x):
    """ Differentiable rounding function
    Input:
        x(tensor)
    Output:
        x(tensor)
    """
    return torch.round(x) + (x - torch.round(x))**3


def quality_to_factor(quality):
    """ Calculate factor corresponding to quality
    Input:
        quality(float): Quality for jpeg compression
    Output:
        factor(float): Compression factor
    """
    if quality < 50:
        quality = 5000. / quality
    else:
        quality = 200. - quality*2
    return quality / 100.


class y_dequantize(nn.Module):
    """ Dequantize Y channel
    Inputs:
        image(tensor): batch x height x width
        factor(float): compression factor
    Outputs:
        image(tensor): batch x height x width

    """
    def __init__(self):
        super(y_dequantize, self).__init__()
        self.y_table = y_table

    def forward(self, image, factor):
        return image * (self.y_table * factor)


class c_dequantize(nn.Module):
    """ Dequantize CbCr channel
    Inputs:
        image(tensor): batch x height x width
        factor(float): compression factor
    Outputs:
        image(tensor): batch x height x width

    """
    def __init__(self):
        super(c_dequantize, self).__init__()
        self.c_table = c_table

    def forward(self, image , factor):
        return image * (self.c_table * factor)


class idct_8x8(nn.Module):
    """ Inverse discrete Cosine Transformation
    Input:
        dcp(tensor): batch x height x width
    Output:
        image(tensor): batch x height x width
    """
    def __init__(self):
        super(idct_8x8, self).__init__()
        alpha = np.array([1. / np.sqrt(2)] + [1] * 7)
        self.alpha = nn.Parameter(torch.from_numpy(np.outer(alpha, alpha)).float())
        tensor = np.zeros((8, 8, 8, 8), dtype=np.float32)
        for x, y, u, v in itertools.product(range(8), repeat=4):
            tensor[x, y, u, v] = np.cos((2 * u + 1) * x * np.pi / 16) * np.cos(
                (2 * v + 1) * y * np.pi / 16)
        self.tensor = nn.Parameter(torch.from_numpy(tensor).float())

    def forward(self, image):

        image = image * self.alpha
        result = 0.25 * torch.tensordot(image, self.tensor, dims=2) + 128
        result.view(image.shape)
        return result


class block_merging(nn.Module):
    """ Merge pathces into image
    Inputs:
        patches(tensor) batch x height*width/64, height x width
        height(int)
        width(int)
    Output:
        image(tensor): batch x height x width
    """
    def __init__(self):
        super(block_merging, self).__init__()

    def forward(self, patches, height, width):
        k = 8
        batch_size = patches.shape[0]
        image_reshaped = patches.view(batch_size, height//k, width//k, k, k)
        image_transposed = image_reshaped.permute(0, 1, 3, 2, 4)
        return image_transposed.contiguous().view(batch_size, height, width)


class chroma_upsampling(nn.Module):
    """ Upsample chroma layers
    Input:
        y(tensor): y channel image
        cb(tensor): cb channel
        cr(tensor): cr channel
    Ouput:
        image(tensor): batch x height x width x 3
    """
    def __init__(self):
        super(chroma_upsampling, self).__init__()

    def forward(self, y, cb, cr):
        def repeat(x, k=2):
            height, width = x.shape[1:3]
            x = x.unsqueeze(-1)
            x = x.repeat(1, 1, k, k)
            x = x.view(-1, height * k, width * k)
            return x

        cb = repeat(cb)
        cr = repeat(cr)

        return torch.cat([y.unsqueeze(3), cb.unsqueeze(3), cr.unsqueeze(3)], dim=3)


class ycbcr_to_rgb_jpeg(nn.Module):
    """ Converts YCbCr image to RGB JPEG
    Input:
        image(tensor): batch x height x width x 3
    Outpput:
        result(tensor): batch x 3 x height x width
    """
    def __init__(self):
        super(ycbcr_to_rgb_jpeg, self).__init__()

        matrix = np.array(
            [[1., 0., 1.402], [1, -0.344136, -0.714136], [1, 1.772, 0]],
            dtype=np.float32).T
        self.shift = nn.Parameter(torch.tensor([0, -128., -128.]))
        self.matrix = nn.Parameter(torch.from_numpy(matrix))

    def forward(self, image):
        result = torch.tensordot(image + self.shift, self.matrix, dims=1)
        #result = torch.from_numpy(result)
        result.view(image.shape)
        return result.permute(0, 3, 1, 2)


class decompress_jpeg(nn.Module):
    """ Full JPEG decompression algortihm
    Input:
        compressed(dict(tensor)): batch x h*w/64 x 8 x 8
        rounding(function): rounding function to use
        factor(float): Compression factor
    Ouput:
        image(tensor): batch x 3 x height x width
    """
    def __init__(self, height, width, rounding=torch.round):
        super(decompress_jpeg, self).__init__()
        self.c_dequantize = c_dequantize()
        self.y_dequantize = y_dequantize()
        self.idct = idct_8x8()
        self.merging = block_merging()
        self.chroma = chroma_upsampling()
        self.colors = ycbcr_to_rgb_jpeg()

        self.height, self.width = height, width

    def forward(self, y, cb, cr , factor):
        components = {'y': y, 'cb': cb, 'cr': cr}
        for k in components.keys():
            if k in ('cb', 'cr'):
                comp = self.c_dequantize(components[k], factor)
                height, width = int(self.height/2), int(self.width/2)
            else:
                comp = self.y_dequantize(components[k], factor)
                height, width = self.height, self.width
            comp = self.idct(comp)
            components[k] = self.merging(comp, height, width)
            #
        image = self.chroma(components['y'], components['cb'], components['cr'])
        image = self.colors(image)

        image = torch.min(255*torch.ones_like(image),
                          torch.max(torch.zeros_like(image), image))
        return image/255


# Standard libraries
import itertools
import numpy as np
# PyTorch
import torch
import torch.nn as nn


class rgb_to_ycbcr_jpeg(nn.Module):
    """ Converts RGB image to YCbCr
    Input:
        image(tensor): batch x 3 x height x width
    Outpput:
        result(tensor): batch x height x width x 3
    """
    def __init__(self):
        super(rgb_to_ycbcr_jpeg, self).__init__()
        matrix = np.array(
            [[0.299, 0.587, 0.114], [-0.168736, -0.331264, 0.5],
             [0.5, -0.418688, -0.081312]], dtype=np.float32).T
        self.shift = nn.Parameter(torch.tensor([0., 128., 128.]))
        #
        self.matrix = nn.Parameter(torch.from_numpy(matrix))

    def forward(self, image):
        image = image.permute(0, 2, 3, 1)
        result = torch.tensordot(image, self.matrix, dims=1) + self.shift
    #    result = torch.from_numpy(result)
        result.view(image.shape)
        return result



class chroma_subsampling(nn.Module):
    """ Chroma subsampling on CbCv channels
    Input:
        image(tensor): batch x height x width x 3
    Output:
        y(tensor): batch x height x width
        cb(tensor): batch x height/2 x width/2
        cr(tensor): batch x height/2 x width/2
    """
    def __init__(self):
        super(chroma_subsampling, self).__init__()

    def forward(self, image):
        image_2 = image.permute(0, 3, 1, 2).clone()
        avg_pool = nn.AvgPool2d(kernel_size=2, stride=(2, 2),
                                count_include_pad=False)
        cb = avg_pool(image_2[:, 1, :, :].unsqueeze(1))
        cr = avg_pool(image_2[:, 2, :, :].unsqueeze(1))
        cb = cb.permute(0, 2, 3, 1)
        cr = cr.permute(0, 2, 3, 1)
        return image[:, :, :, 0], cb.squeeze(3), cr.squeeze(3)


class block_splitting(nn.Module):
    """ Splitting image into patches
    Input:
        image(tensor): batch x height x width
    Output:
        patch(tensor):  batch x h*w/64 x h x w
    """
    def __init__(self):
        super(block_splitting, self).__init__()
        self.k = 8

    def forward(self, image):
        height, width = image.shape[1:3]
        batch_size = image.shape[0]
        image_reshaped = image.view(batch_size, height // self.k, self.k, -1, self.k)
        image_transposed = image_reshaped.permute(0, 1, 3, 2, 4)
        return image_transposed.contiguous().view(batch_size, -1, self.k, self.k)


class dct_8x8(nn.Module):
    """ Discrete Cosine Transformation
    Input:
        image(tensor): batch x height x width
    Output:
        dcp(tensor): batch x height x width
    """
    def __init__(self):
        super(dct_8x8, self).__init__()
        tensor = np.zeros((8, 8, 8, 8), dtype=np.float32)
        for x, y, u, v in itertools.product(range(8), repeat=4):
            tensor[x, y, u, v] = np.cos((2 * x + 1) * u * np.pi / 16) * np.cos(
                (2 * y + 1) * v * np.pi / 16)
        alpha = np.array([1. / np.sqrt(2)] + [1] * 7)
        #
        self.tensor =  nn.Parameter(torch.from_numpy(tensor).float())
        self.scale = nn.Parameter(torch.from_numpy(np.outer(alpha, alpha) * 0.25).float() )

    def forward(self, image):
        image = image - 128
        result = self.scale * torch.tensordot(image, self.tensor, dims=2)
        result.view(image.shape)
        return result


class y_quantize(nn.Module):
    """ JPEG Quantization for Y channel
    Input:
        image(tensor): batch x height x width
        rounding(function): rounding function to use
        factor(float): Degree of compression
    Output:
        image(tensor): batch x height x width
    """
    def __init__(self, rounding):
        super(y_quantize, self).__init__()
        self.rounding = rounding
        self.y_table = y_table

    def forward(self, image, factor):
        image = image.float() / (self.y_table * factor)
        image = self.rounding(image)
        return image


class c_quantize(nn.Module):
    """ JPEG Quantization for CrCb channels
    Input:
        image(tensor): batch x height x width
        rounding(function): rounding function to use
        factor(float): Degree of compression
    Output:
        image(tensor): batch x height x width
    """
    def __init__(self, rounding):
        super(c_quantize, self).__init__()
        self.rounding = rounding
        self.c_table = c_table

    def forward(self, image, factor):
        image = image.float() / (self.c_table * factor)
        image = self.rounding(image)
        return image


class compress_jpeg(nn.Module):
    """ Full JPEG compression algortihm
    Input:
        imgs(tensor): batch x 3 x height x width
        rounding(function): rounding function to use
        factor(float): Compression factor
    Ouput:
        compressed(dict(tensor)): batch x h*w/64 x 8 x 8
    """
    def __init__(self, rounding=torch.round):
        super(compress_jpeg, self).__init__()
        self.l1 = nn.Sequential(
            rgb_to_ycbcr_jpeg(),
            chroma_subsampling()
        )
        self.l2 = nn.Sequential(
            block_splitting(),
            dct_8x8()
        )
        self.c_quantize = c_quantize(rounding=rounding)
        self.y_quantize = y_quantize(rounding=rounding)

    def forward(self, image, factor=1):
        y, cb, cr = self.l1(image*255)
        components = {'y': y, 'cb': cb, 'cr': cr}
        for k in components.keys():
            comp = self.l2(components[k])
            if k in ('cb', 'cr'):
                comp = self.c_quantize(comp, factor)
            else:
                comp = self.y_quantize(comp, factor)

            components[k] = comp

        return components['y'], components['cb'], components['cr']

class DiffJPEG(nn.Module):
    def __init__(self, height, width, differentiable=True):
        ''' Initialize the DiffJPEG layer
        Inputs:
            height(int): Original image hieght
            width(int): Original image width
            differentiable(bool): If true uses custom differentiable
                rounding function, if false uses standrard torch.round
            quality(float): Quality factor for jpeg compression scheme.
        '''
        super(DiffJPEG, self).__init__()
        if differentiable:
            rounding = diff_round
        else:
            rounding = torch.round
        self.compress = compress_jpeg(rounding=rounding)
        self.decompress = decompress_jpeg(height, width, rounding=rounding)

    def forward(self, x, quality=80):
        '''

        '''

        factor = quality_to_factor(quality)
        if factor == 0:
          return x

        y, cb, cr = self.compress(x, factor)
        recovered = self.decompress(y, cb, cr, factor)
        return recovered

In [24]:
SEED = hyperparams['SEED']
strength = hyperparams['STRENGTH']
guidance_scale = hyperparams['GUIDANCE_SCALE']
num_inference_steps = hyperparams['NUM_DENOISING_STEPS']
num_images = len(image_url)
image_tensor = torch.zeros((num_images,3,512,512)).half()
loss = []

#Save individual image tensors
save_image_tensor = f'/kaggle/working/ImageTensor'
if not os.path.exists(save_image_tensor):
    os.mkdir(save_image_tensor)

height, width = (512,512)  #HardCoded
jpeg = DiffJPEG(height=height, width=width, differentiable=True).to('cuda')  #Change

for i in range(num_images):
    init_seeds(SEED)
    prompt_guidance = hyperparams['PROMPT_ATTACK']
    init_image = download_image(image_url[i])
    if init_image == None:
        continue
    X = preprocess(init_image).half().to("cuda")
    start = time.time()
    result, last_imag, loss_sample = super_linf(X,
                      jpeg_model = jpeg,
                      prompt=prompt_guidance,
                      eps=hyperparams['EPS'],
                      step_size=hyperparams['STEP_SIZE'],
                      iters=hyperparams['ITERS'],
                      clamp_min = hyperparams['CLAMP_MIN'],
                      clamp_max = hyperparams['CLAMP_MAX'],
                      num_inference_steps=num_inference_steps,
                      guidance_scale=guidance_scale,
                      grad_reps=hyperparams['GRAD_REPS']
                     )
    torch.save(result.cpu(), f"{save_image_tensor}/{i+offset}_result_image.pt")
    total_time = time.time() - start
    image_tensor[i, :, :, :] = result.cpu()
    loss.append(loss_sample)
    del result, last_imag, X
    torch.cuda.empty_cache()
    gc.collect()

AVG Loss: 553.188: 100%|██████████| 1/1 [00:12<00:00, 12.48s/it]


Error downloading/processing image: cannot identify image file <_io.BytesIO object at 0x7e0b376291c0>


AVG Loss: 401.562: 100%|██████████| 1/1 [00:12<00:00, 12.92s/it]


Error downloading/processing image: cannot identify image file <_io.BytesIO object at 0x7e0c8183d490>


  0%|          | 0/1 [00:02<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
to_pil = T.ToPILImage()
model_id_or_path = "runwayml/stable-diffusion-v1-5"
# model_id_or_path = "CompVis/stable-diffusion-v1-4"
# model_id_or_path = "CompVis/stable-diffusion-v1-3"
# model_id_or_path = "CompVis/stable-diffusion-v1-2"
# model_id_or_path = "CompVis/stable-diffusion-v1-1"
os.environ['HF_HUB_OFFLINE']='True'

pipe_img2img = StableDiffusionImg2ImgPipeline.from_pretrained(
    model_id_or_path,
    revision="fp16",
    safety_checker=None,
    torch_dtype=torch.float16,
    cache_dir = "/content/drive/MyDrive/MajorProject/Diffusers_cache_test/"
)
pipe_img2img = pipe_img2img.to('cuda')

In [ ]:
import json
for i in range(num_images):
    result = image_tensor[i].unsqueeze(0).cuda()
    adv_X = (result / 2 + 0.5).clamp(0, 1)    
    adv_image = to_pil(adv_X[0]).convert("RGB")
    num_inference_steps = hyperparams['NUM_DENOISING_STEPS_FINAL']
    prompt_image = prompt[i]
    torch.cuda.empty_cache()
    
    path_to_save = f'/kaggle/working/{i+offset}/'
    if not os.path.exists(path_to_save):
        os.mkdir(path_to_save)
    
    init_image = download_image(image_url[i])
    if init_image == None:
        continue
    gen_image(init_image, adv_image, path_to_save, prompt_image, QF)
    save_dict_to_text_file( hyperparams, path_to_save, total_time, loss)


In [ ]:
!zip -r fileensemblediffusion10.zip /kaggle/working
from IPython.display import FileLink
FileLink(r'fileensemblediffusion10.zip')